# Imports

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_community.document_loaders import DirectoryLoader

from langchain_community.document_loaders import PyPDFLoader

from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

from langchain_community.vectorstores import Chroma

import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain_community.llms import HuggingFacePipeline

from langchain.chains import RetrievalQA

/Users/isaac/Documents/Programming/AI/ChatbotAI-D2/.venv/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


# Fetch Source Docs

## pdf loader

In [2]:
loader = DirectoryLoader('./Source Docs/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [3]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [4]:
texts[1]

Document(page_content="Page 1 of 24https://www.destinypedia.com/Lore:Ghost_Stories\nSign up Login\nNavigationGames\nOther media\nGameplay\nThe universe\nLore database\nCommunity InfoDestinypedia\nWiki help\nRelated sites\nBungie.netRecent changes Random pageSpecial pages What links hereDid you know?...that the Red Legion were feared across the galaxy as destroyers of worlds?...that the Dreadnaught was partially constructed out of a part of Akka's corpse?...that Zhalo Supercell was originally named after the hammer of the Finnish thunder god Ukko?...that while Mercury was shown to only be partially converted into a machine world during theDark Age in Season of Dawn, it was previously stated in Destiny that it had been converted in amatter of days?...that Ghost was originally voiced by Peter Dinklage, but his lines were re-recorded by Nolan Northand completely replaced with Destiny patch 2.0.0?Lore Discussion View source HistoryLore:Ghost StoriesFrom Destinypedia, the Destiny wiki\nDisco

## url

In [5]:
# use huggingface instructor embeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl", model_kwargs={"device": "cpu"})


load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
# Generate and store embeddings in Chroma
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=instructor_embeddings,
                                 persist_directory=persist_directory)
vectordb.persist()

In [7]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

device_map = {
    "transformer.word_embeddings": 0,
    "transformer.word_embeddings_layernorm": 0,
    "lm_head": "cpu",
    "transformer.h": 0,
    "transformer.ln_f": 0,
    "dtype": "float32"
}

quantization_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/wizardLM-7B-HF")

model = LlamaForCausalLM.from_pretrained("TheBloke/wizardLM-7B-HF", device_map=device_map, quantization_config=quantization_config)                                          

OSError: TheBloke/WizardLM-30B-Uncensored-GPTQ does not appear to have a file named pytorch_model.bin, tf_model.h5, model.ckpt or flax_model.msgpack.

In [ ]:
pipe = pipeline(
    "text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=local_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
# full example
query = "Who is Clovis?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'